**목표**

용봉은 승촌보 인근에 위치하고 있는데, 조류 발생 문제가 빈번하게 발생한다.
광주에서 나오는 하수처리장의 방류수가 하천 유량의 70% 정도가 되어 수질이 매우 안 좋으며
녹조도 많이 발생한다.

1. 조류 발생과 관련하여 조류(클로로필) 농도가 높게 나타나는 계절을 찾는다. 주로 여름철?
2. 조류 발생 관련 인자로 총질소, 총인, 온도 등 중에 어느 요인이 가장 중요한지 찾는다.
    현재 국내 정책은 총인의 농도가 가장 중요하다고 여기고 정책을 진행한다.
3. 보 건설에 따른 영향을 보고 싶지만 보 건설은 2011년에 했고, 
    수질 데이터는 2012년부터 공개가 되었어서 보 건설 전후로 설명은 어렵다.
    다만, 2017년부터 보개방을 시작했기에 수문 개방에 따른 수질 효과 분석은 가능하다.
4. 승촌보(용봉) 데이터와 하류에 있는 죽산보(나주) 측정망 자료를 이용하여
    상류와 하류의 비교도 진행한다.

**필수 라이브러리**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

plt.rc('figure', figsize=(10, 6))  # 그림 크기 설정

**matplotlib 한글 설정**

- 운영체제에 따른 한글 지원 설정. 윈도우, 우분투, 구글 코랩 지원.
- 참고: [matplotlib에서 한글 지원하기](https://github.com/codingalzi/datapy/blob/master/matplotlib-korean.md)

In [2]:
import platform

if platform.system() == 'Windows': # 윈도우
    from matplotlib import font_manager, rc
    font_path = "C:/Windows/Fonts/NGULIM.TTF"
    font = font_manager.FontProperties(fname=font_path).get_name()
    rc('font', family=font)
elif platform.system() == 'Linux': # 우분투 또는 구글 코랩
    # !sudo apt-get install -y fonts-nanum*
    # !fc-cache -fv
    
    applyfont = "NanumBarunGothic"
    import matplotlib.font_manager as fm
    if not any(map(lambda ft: ft.name == applyfont, fm.fontManager.ttflist)):
        fm.fontManager.addfont("/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf")
    plt.rc("font", family=applyfont)
    plt.rc("axes", unicode_minus=False)
    

**데이터**

데이터는 연도별로 분리되어 있으며 원본 엑셀 파일은 아래 서버에 저장되어 있다.

In [3]:
base_url = "https://github.com/codingalzi/water-data/raw/master/reservoirs/"

**승촌보 (엑셀) 자료를 데이터프레임으로 불러오기**

- `skiprows=0`: 0번 행 무시. 모든 워크시트 대상.
- `header=1`: 1번 행을 header로 지정, 즉 열 인덱스로 사용.
- `sheet_name=None`: 모든 워크시트 가져오기. 워크시트별로 하나의 df 생성. 반환값은 사전.
- `na_values='ND'`: ND(No Data/Not Determinded) 또한 결측치로 처리. 용존산소 열에 포함되어 있음.
- `index_col=0`: 측정일을 행 인덱스로 사용
- `parse_dates=True`: 행 인덱스로 사용되는 날짜 대상 파싱 실행

주의: 아래 모듈을 먼저 설치해야 할 수도 있다.

```python
!pip install openpyxl
```

In [4]:
scb_yearly = pd.read_excel(base_url+"Seungchonbo.xlsx", 
                            skiprows=0, 
                            header=1, 
                            sheet_name=None,
                            na_values='ND',
                            index_col=0, 
                            parse_dates=True)

In [15]:
scb_yearly_flow_rate = pd.read_excel("C:/Users/gslee/Documents/"+"Seungchonbo-flow_rate.xlsx", 
                            header=0, 
                            sheet_name=None,
                            na_values='ND',
                            index_col=0, 
                            parse_dates=True)

In [16]:
scb_yearly_flow_rate.keys()

dict_keys(['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'])

In [18]:
scb_yearly["2015년"]

,측정소명,수온 ℃,수소이온농도,전기전도도 μS/cm,용존산소 mg/L,총유기탄소 mg/L,총질소 mg/L,총인 mg/L,클로로필-a mg/㎥
측정일/월,,,,,,,,,
2015-01-01 00:00:00,용봉,6.3,7.0,410.0,9.2,2.5,5.607,0.057,3.9
2015-01-01 01:00:00,용봉,6.2,7.0,410.0,9.2,2.5,5.596,0.056,3.9
2015-01-01 02:00:00,용봉,6.1,7.0,409.0,9.2,2.4,5.485,0.053,3.7
2015-01-01 03:00:00,용봉,6.1,7.0,410.0,9.2,2.4,5.487,0.052,3.7
2015-01-01 04:00:00,용봉,6.0,7.0,412.0,9.3,2.4,5.625,0.052,3.8
...,...,...,...,...,...,...,...,...,...
2015-12-31 19:00:00,용봉,7.8,6.9,399.0,9.0,3.2,6.088,0.062,9.8
2015-12-31 20:00:00,용봉,7.7,6.9,401.0,9.0,3.3,6.052,0.056,9.4
2015-12-31 21:00:00,용봉,7.7,6.9,401.0,9.0,3.4,6.152,0.053,9.6


In [23]:
scb_yearly_flow_rate["2020"]

,상류수위(EL.m),하류수위(EL.m),유입량(㎥/s),방류량(㎥/s)
시간,,,,
2020.01.01 00:00,6.00,1.54,13.20,13.20
2020.01.01 00:10,6.00,1.54,13.20,13.20
2020.01.01 00:20,6.00,1.54,12.09,9.86
2020.01.01 00:30,6.00,1.54,10.75,9.20
2020.01.01 00:40,6.00,1.56,9.42,9.20
...,...,...,...,...
2020.12.31 23:10,6.08,1.60,29.64,13.20
2020.12.31 23:20,6.08,1.59,21.42,13.20
2020.12.31 23:30,6.07,1.59,13.20,13.20


In [24]:
scb_yearly_flow_rate["2016"].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52703 entries, 2016-01-01 00:00:00 to 2016-12-31 23:50:00
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0    상류수위(EL.m)  52703 non-null  float64
 1    하류수위(EL.m)  52703 non-null  float64
 2    유입량(㎥/s)    52703 non-null  float64
 3    방류량(㎥/s)    52703 non-null  float64
dtypes: float64(4)
memory usage: 2.0 MB


In [25]:
scb_yearly_flow_rate["2016"].describe()

,상류수위(EL.m),하류수위(EL.m),유입량(㎥/s),방류량(㎥/s)
count,52703.000000,52703.000000,52703.000000,52703.000000
mean,7.511977,3.595928,34.302987,34.274017
std,0.088400,0.124143,59.973405,60.849639
min,6.880000,0.000000,0.000000,0.000000
25%,7.490000,3.550000,14.810000,14.790000
50%,7.510000,3.570000,20.010000,20.140000
75%,7.550000,3.630000,30.110000,29.650000
max,7.920000,5.090000,1045.250000,1072.350000
